In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing the train.csv

#### Importing Data and basic EDA

In [ ]:
data = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")

In [ ]:
data.shape
data.head(10)

In [ ]:
data.describe()

In [ ]:
data.shape

Creating a copy of data to use in future

In [ ]:
df = data.copy()

In [ ]:
df2 = data.copy()

Importing the Test.csv


In [ ]:
test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")

In [ ]:
test.shape

In [ ]:
type(test)

In [ ]:
data.head()

Name of all the columns present in the train dataset

In [ ]:
data.columns

finding how many columns have null values

In [ ]:
columns_with_null = data.isnull().sum()  
columns_with_null = columns_with_null[columns_with_null > 0]  
columns_with_null.shape

In [ ]:
target_columns = 'target'

Numerical columns:

In [ ]:
numerical_columns = data.select_dtypes(include=['float64', 'int64']) 
numerical_columns.columns

In [ ]:
categorical_columns = data.select_dtypes(include=['category', 'object']) 
categorical_columns.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8,5))
target_column = data['target'].value_counts()
plt.pie(target_column, labels = target_column.index, autopct='%1.1f%%')
plt.title('Target Distribution')
plt.show()

The distribution of target varible is almost equal making it balanced
* 1 (which means Malware was detected) is about 50.5% of the total systems
* 0 (which means Malware was not detected) is about 49.5% of the total systems

finidng correlation between target column and numerical columns and putting them in ascending order

In [ ]:
numerical_df = data.select_dtypes(include=['float64', 'int64'])
corr_matrix = numerical_df.corr()
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

Observing Stats details of numerical columns

In [ ]:
data.describe()

#### Plotting graphs to identity patterns

In [ ]:
data['NumAntivirusProductsInstalled'].value_counts()

In [ ]:
plt.figure(figsize=(8,5))
antivirusinstalled = pd.crosstab(data['NumAntivirusProductsInstalled'], data['target'], normalize = 'index')*100
ax = antivirusinstalled.plot(kind = 'bar')
for container in  ax.containers:
    ax.bar_label(container, fmt= '%1.1f%%', label_type = 'edge')
plt.show()

- More number of AntiVirus Installed reduces the chances of system threat

In [ ]:
data['NumAntivirusProductsEnabled'].value_counts()

In [ ]:
plt.figure(figsize=(8,5))
antivirusinstalled = pd.crosstab(data['NumAntivirusProductsEnabled'], data['target'], normalize = 'index')*100
ax = antivirusinstalled.plot(kind = 'bar')
for container in  ax.containers:
    ax.bar_label(container, fmt= '%1.1f%%', label_type = 'edge')
plt.show()

In [ ]:
data['IsAlwaysOnAlwaysConnectedCapable'].value_counts()

In [ ]:
plt.figure(figsize=(8,5))
antivirusinstalled = pd.crosstab(data['IsAlwaysOnAlwaysConnectedCapable'], data['target'], normalize = 'index')*100
ax = antivirusinstalled.plot(kind = 'bar')
for container in  ax.containers:
    ax.bar_label(container, fmt= '%1.1f%%', label_type = 'edge')
plt.show()

 - There is slightly less chance of system threath if system is always connected to a cable

In [ ]:
data['RealTimeProtectionState'].value_counts()

In [ ]:
data['OSVersion'].value_counts()

In [ ]:
data['OSInstallType'].value_counts()

### Milestone 1

In [ ]:
data['OSVersion'].unique()

In [ ]:
data['NumAntivirusProductsInstalled'].max()

In [ ]:
data['IsGamer']

In [ ]:
gamer_malware = data[(data['IsGamer']== 1) & (data['target']== 1)]
count = len(gamer_malware)
count

In [ ]:
passive_mode = data[data['IsPassiveModeEnabled'] == 1]
passive_mode['RealTimeProtectionState'].mode()[0]

In [ ]:
data[(data['PrimaryDisplayResolutionHorizontal'] == 1366) & (data['PrimaryDisplayResolutionVertical']== 768)].shape[0]

In [ ]:
data['TotalPhysicalRAMMB'].describe()

#### Dealing with nan values

In [ ]:
total_null = data.isnull().sum().sum()
total_entries = data.size
null_percentage = (total_null / total_entries) * 100
null_percentage

By checking out the data manually, it is found to have values like UNKNOWN, Unknown and unknown. I replaced these with Nan Values

In [ ]:
data.replace({'unknown': np.nan, 'Unknown': np.nan, 'UNKNOWN': np.nan}, inplace=True)

Doing the same for test dataset

In [ ]:
test.replace({'unknown': np.nan, 'Unknown': np.nan, 'UNKNOWN': np.nan}, inplace=True)

In [ ]:
data.isna().sum().sum()

In [ ]:
data.shape

Using simple imputer to impute the nan values

In [ ]:
from sklearn.impute import SimpleImputer

num_cols = data.select_dtypes(include=['int64', 'float64']).columns
cat_cols = data.select_dtypes(include=['object', 'category']).columns

num_imputer = SimpleImputer(strategy='most_frequent')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

using the same for test dataset to maintain consistency

In [ ]:
num_cols2 = test.select_dtypes(include=['int64', 'float64']).columns
cat_cols2 = test.select_dtypes(include=['object', 'category']).columns

num_imputer = SimpleImputer(strategy='most_frequent')
test[num_cols2] = num_imputer.fit_transform(test[num_cols2])

cat_imputer = SimpleImputer(strategy='most_frequent')
test[cat_cols2] = cat_imputer.fit_transform(test[cat_cols2])

In [ ]:
data.isna().sum().sum()

In [ ]:
test.isna().sum().sum()

### Milestone 2

In [ ]:
df_copy = df.copy()

In [ ]:
df_copy2 = df.copy()

In [ ]:
df.head()

In [ ]:
constant_columns = data.nunique()[data.nunique() == 1]
print(constant_columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#selecting categorical columns as a list and getting names of the columns
categorical_cols = df.select_dtypes(include=['object']).columns

encoder = LabelEncoder()
#looping through the list and fit and transform on all cols in categorical cols
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])
corr_matrix = df.corr()
corr_pairs = corr_matrix.abs().unstack()
corr_pairs = corr_pairs[corr_pairs < 1].sort_values(ascending=False)
corr_pairs.head(1)

Create a new dataframe called cat_df which contains only the columns of datatype  'object'. In cat_df, for all the columns which take less than or equal to 10 unique values, use OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'). What is the new number of columns in the cat_df dataframe?

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_df = df_copy.select_dtypes(include=['object'])
low_cardinality_cols = cat_df.nunique()[cat_df.nunique() <= 10].index

encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
encoded_cols = encoder.fit_transform(cat_df[low_cardinality_cols])

encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(low_cardinality_cols))

cat_df = cat_df.drop(columns=low_cardinality_cols)
cat_df = pd.concat([cat_df, encoded_df], axis=1)
cat_df.shape[1]

Create a new dataframe called num_df which contains only the columns of datatype  'int64' and 'float64'. Use MinMaxScaler() on num_df. What is the sum of all the values in num_df?

In [ ]:
from sklearn.preprocessing import MinMaxScaler

num_df = df_copy.select_dtypes(include=['int64', 'float64'])

scaler = MinMaxScaler()
scaled_num_df = pd.DataFrame(scaler.fit_transform(num_df), columns=num_df.columns)
sum_values = scaled_num_df.sum().sum()
print(sum_values)

In [ ]:
df.isna().sum().sum()

Use the original data (X,y) for this question. Apply the following steps on X:
1. Use SimpleImputer to fill missing values with strategy as 'most_frequent'
2. For all columns of datatype 'object', use OrdinalEncoder()
3. Apply train_test_split with test_size = 0.2 and random_state = 42
4. Train SGDClassifier with default parameters and random_state = 42

What is the accuracy score received on the test data from the train_test_split?


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

imputer = SimpleImputer(strategy='most_frequent')
df_copy2 = pd.DataFrame(imputer.fit_transform(df_copy2), columns=df_copy2.columns)

cat_cols = df_copy2.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder()
df_copy2[cat_cols] = encoder.fit_transform(df_copy2[cat_cols])

X = df_copy2.drop(columns=['target']) 
y = df_copy2['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sgd_model = SGDClassifier(random_state=42)
sgd_model.fit(X_train, y_train)

y_pred = sgd_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

#### Spliting the Dataset into test and train 

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(columns=['target']) 
y = data['target'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### Encoding and Scaling

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

X_train = X_train.copy()
X_test = X_test.copy()

num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=[object, 'category']).columns.tolist()


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_cols)  
    ],
    remainder='passthrough' 
)

In [ ]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)
test = preprocessor.transform(test)

all_columns = cat_cols + num_cols 
X_train = pd.DataFrame(X_train, columns=all_columns)
X_test = pd.DataFrame(X_test, columns=all_columns)
test = pd.DataFrame(test, columns=all_columns)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)

X_train = pd.DataFrame(X_train, columns=all_columns)
X_test = pd.DataFrame(X_test, columns=all_columns)
test = pd.DataFrame(test, columns=all_columns)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
test

#### Trying out different Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
clf = LogisticRegression() 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy1 = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy1)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy2 = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy2)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

params = {
    'objective': 'binary:logistic',  
    'eval_metric': 'logloss',  
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy3 = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy3)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    'objective': 'binary',  
    'metric': 'binary_logloss',  
    'boosting_type': 'gbdt', 
    'max_depth': -1,  
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42
}

model2 = lgb.LGBMClassifier(**params)
model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)

accuracy4 = accuracy_score(y_test, y_pred)
print("Accuracy Score: ", accuracy4)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
model_accuracies = {
    "Logistic Regression": accuracy,
    "KNN" : accuracy1,
    "Random Forest": accuracy2,
    "XGBClassifier": accuracy3,
    "LGBMClassifier": accuracy4
}

model_accuracies

In [ ]:
acc_df = pd.DataFrame(list(model_accuracies.items()), columns=["Model", "Accuracy"])
acc_df = acc_df.sort_values(by="Accuracy", ascending=False)  
print(acc_df)

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.barplot(x=list(model_accuracies.keys()), y=list(model_accuracies.values()), palette="viridis")
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.ylim(0, 1) 
plt.xticks(rotation=45)
plt.show()

### Hyper-Parameter Tuning Using RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rf= RandomForestClassifier()
param_dist = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30], 
    'min_samples_split': [2, 5, 10],  
}

random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist, 
    n_iter=10, scoring='accuracy', 
    cv=3, verbose=1, random_state=42, n_jobs=-1
)

random_search.fit(X_train, y_train)


In [ ]:
random_search.best_params_

In [ ]:
rf_tuned = random_search.best_estimator_
y_pred = rf_tuned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
xgb_tuned = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
param_dist = {
    'n_estimators': [50, 100, 200],  
    'max_depth': [3, 6, 9], 
    'learning_rate': [0.01, 0.05, 0.1] 
}

random_search = RandomizedSearchCV(
    xgb_tuned, param_distributions=param_dist, 
    n_iter=100, scoring='accuracy', 
    cv=3, verbose=1, random_state=42, n_jobs=-1
)


random_search.fit(X_train, y_train)


In [ ]:
xgb_tuned = random_search.best_estimator_
y_pred = xgb_tuned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

random_search.best_params_
accuracy

In [ ]:
lgbm_model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', random_state=42)

param_dist = {
    'n_estimators': [50, 100, 200], 
    'max_depth': [-1, 5, 10], 
    'learning_rate': [0.01, 0.05, 0.1] 
}

random_search = RandomizedSearchCV(
    lgbm_model, param_distributions=param_dist, 
    n_iter=100, scoring='accuracy', 
    cv=3, verbose=1, random_state=42, n_jobs=-1
)

random_search.fit(X_train, y_train)

In [ ]:
lgbm_tuned = random_search.best_estimator_
y_pred = lgbm_tuned.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
random_search.best_params_

### Milestone 3

Fit a Decision tree model (random state 42) on the training set and perform hyper parameter tuning using grid search with 3 folds and use scoring as accuracy, using the following values:

max_depth: [20, 30]
min_samples_split: [2, 5]
min_samples_leaf: [1, 2]

Trying GRIDSEARCH For DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dt = DecisionTreeClassifier(random_state=42)

param_grid1 = {
    'max_depth': [20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(
    estimator=dt,
    param_grid=param_grid1,
    cv=3, 
    scoring='accuracy',   
)

grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
best_params

In [ ]:
clf = DecisionTreeClassifier(**best_params)  
clf.fit(X_train, y_train) 

accuracy = clf.score(X_test, y_test)
accuracy

Fit an AdaBoostClassifier model (random state 42) on the training set and perform hyper parameter tuning using grid search with 3 folds and use scoring as accuracy, using the following values :

a. n_estimators: [10, 20, 30]

b. learning_rate: [5, 10]

c. algorithm: ['SAMME']

Trying GRIDSEARCH For ADABOOSTCLASSIFIER

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier( random_state=42)
param_grid2 = {
    'n_estimators': [10, 20, 30],    
    'learning_rate': [5, 10],        
    'algorithm': ['SAMME']           
}

grid_search = GridSearchCV(
    estimator=adaboost,
    param_grid=param_grid2,
    scoring='accuracy',  
    cv=3,  
)

grid_search.fit(X_train, y_train)


In [ ]:
best_params2 = grid_search.best_params_
best_params2

In [ ]:
ada = AdaBoostClassifier(**best_params2)  
ada.fit(X_train, y_train) 

accuracy = ada.score(X_test, y_test)
accuracy

### Experimenting with PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)  
X_test_pca = pca.transform(X_test) 

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, marker='o', linestyle='--')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.title('Scree Plot')
plt.show()

1. Elbow Point for Optimal Components
   - The variance drops steeply at first, then levels off.  
   - The point where the slope changes suggests that keeping 10-15 components is sufficient to capture most of the variance.  


2. Diminishing Returns 
   - The first few components explain most variance.  
   - After 20 components, adding more doesn't contribute much.  



In [ ]:
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance Plot')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
plt.axhline(y=0.90, color='g', linestyle='--', label='90% Variance')
plt.axhline(y=0.85, color='orange', linestyle='--', label='85% Variance')
plt.legend()
plt.grid(True)
plt.show()


* for simpler model with good compression, 25-30 PCs (85% variance) is enough.
* for high accuracy with minimal loss, 30-35 PCs (90% variance) is ideal.
* for preserve almost all information, 40-45 PCs (95% variance) is necessary 


In [ ]:
pca = PCA(n_components = 0.9)
X_train_pca_90 = pca.fit_transform(X_train)  
X_test_pca_90 = pca.transform(X_test) 

In [ ]:
X_train_pca_90

While using PCA with 90% variance retention, Accuracy decreases :(

In [ ]:
lgbm_tuned.fit(X_train_pca_90, y_train)

y_pred = lgbm_tuned.predict(X_test_pca_90)

accuracy4 = accuracy_score(y_test, y_pred)
accuracy4

Using PCA also decreases the accuracy for XGBClassifier

In [ ]:
xgb_tuned.fit(X_train_pca_90, y_train)
y_pred = xgb_tuned.predict(X_test_pca_90)
accuracy2 = accuracy_score(y_test, y_pred)
accuracy2

### Milestone 4

The preprocessing steps mentioned below are to be used for all the questions that are a part of this milestone
Drop the rows with missing values from your training data
Separate the features(X) and the target vector(y)
Encode the columns of type 'object' or 'category' using ordinal encoder
Scale the entire dataset using StandardScaler

In [ ]:
df2.head()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
le = LabelEncoder()

cat_cols = df2.select_dtypes(include=['object']).columns
num_cols = df2.select_dtypes(include=['int32', 'int64', 'float64']).columns

#imputing
mean_imp = SimpleImputer(strategy='mean')  
mode_imp = SimpleImputer(strategy='most_frequent') 

df2[num_cols] = mean_imp.fit_transform(df2[num_cols]) 
df2[cat_cols] = mode_imp.fit_transform(df2[cat_cols]) 

#label encoding on categorical columns
for col in cat_cols:
    df2[col] = le.fit_transform(df2[col]) 

In [ ]:
num_cols = df2.select_dtypes(include=['int32', 'int64', 'float64']).columns.drop('target', errors='ignore')
scaler = StandardScaler()
df2[num_cols] = scaler.fit_transform(df2[num_cols])

In [ ]:
df2.head()

In [ ]:
X = df2.drop(columns=['target'])  
y = df2['target']
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pca2 = PCA()  
X_pca = pca2.fit_transform(df2)

In [ ]:
type(X_pca)

What is the number of components required to explain 70% of the variance?

In [ ]:
cumulative_variance = np.cumsum(pca2.explained_variance_ratio_)
n_70 = np.argmax(cumulative_variance >= 0.70) + 1
n_70

In [ ]:
cumulative_variance.shape

Reconstruct the data using the number of components obtained in the previous question and enter the Mean Squared Error value obtained

In [ ]:
from sklearn.metrics import mean_squared_error
pca_70 = PCA(n_components=n_70)
X_reduced = pca_70.fit_transform(df2)
X_reconstructed = pca_70.inverse_transform(X_reduced)
mse = mean_squared_error(df2, X_reconstructed)
mse

How much variance is explained by 40 components?

In [ ]:
cumulative_variance = np.cumsum(pca2.explained_variance_ratio_)
explained_variance_40 = cumulative_variance[39]
explained_variance_40

Select top 15 features from the train dataset using f_classif as the score function

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_classif

constant_filter = VarianceThreshold(threshold=0)  # Remove features with zero variance
X_train_filtered = constant_filter.fit_transform(X_train)
filtered_features = X_train.columns[constant_filter.get_support()]
filtered_features   #retained columns after remvoing

In [ ]:
selector = SelectKBest(k=15)
X_train_selected = selector.fit_transform(X_train_filtered, y_train)
selected_features = filtered_features[selector.get_support()]
selected_features

What is the score of the best feature as obtained?

In [ ]:
f_scores = selector.scores_
best_feature_score = max(f_scores)
best_feature_score

In [ ]:
best_feature_index = f_scores.argmax()
best_feature = filtered_features[best_feature_index]
best_feature

Make use of different feature selection methods and describe your process and findings. 
Fit Lasso and Ridge models and provide your insights on the values of the coefficients obtained

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

model = LinearRegression()

rfe = RFE(estimator=model, n_features_to_select=15)
X_train_selected = rfe.fit_transform(X_train, y_train)
X_test_selected = rfe.transform(X_test)
selected_features = X_train.columns[rfe.support_]
selected_features

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X_train_selected, y_train)

y_pred_lasso = lasso.predict(X_test_selected)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_lasso

In [ ]:
lasso_coeffs = pd.Series(lasso.coef_, index=selected_features)
lasso_coeffs.sort_values(ascending = False)

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(X_train_selected, y_train)
y_pred_ridge = ridge.predict(X_test_selected)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mse_ridge

In [ ]:
ridge_coeffs = pd.Series(ridge.coef_, index=selected_features)
ridge_coeffs.sort_values(ascending = False)

In [ ]:
y_pred2 = model2.predict(test)


In [ ]:
y_pred2.shape

In [ ]:
submission = pd.DataFrame({"id" : range(0,test.shape[0]),
                          "target" : y_pred2})
submission.to_csv('submission.csv' ,index = False)